<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#1.-대형마트-(MT1)" data-toc-modified-id="1.-대형마트-(MT1)-1">1. 대형마트 (MT1)</a></span><ul class="toc-item"><li><span><a href="#1.1-단독다가구" data-toc-modified-id="1.1-단독다가구-1.1">1.1 단독다가구</a></span></li><li><span><a href="#1.2-연립" data-toc-modified-id="1.2-연립-1.2">1.2 연립</a></span></li><li><span><a href="#1.3-오피스텔" data-toc-modified-id="1.3-오피스텔-1.3">1.3 오피스텔</a></span></li></ul></li><li><span><a href="#2.-데이터-통합" data-toc-modified-id="2.-데이터-통합-2">2. 데이터 통합</a></span></li></ul></div>

In [29]:
import requests
from urllib.parse import urlparse
import numpy as np
import pandas as pd
import geopandas
from haversine import haversine
import time
from tqdm import tqdm

import warnings
warnings.filterwarnings(action='ignore')

https://developers.kakao.com/docs/latest/ko/local/dev-guide#search-by-category
- 카카오 developers > 개발 가이드 > 카테고리로 장소 검색


- 대형마트 : MT1 (수진)
- 편의점 : CS2 (기원)
- 문화시설 : CT1 (서영)
- 카페 : CE7 (서영)
- 병원 : HP8 (기원)
- 음식점 : FD6 (수진)(패스트푸드만 분류해야 함)

# 1. 대형마트 (MT1)

## 1.1 단독다가구

In [46]:
data = pd.read_csv('data/단독다가구_위도경도.csv')
print(data.shape)
data.head()

(12342, 3)


,도로명,위도,경도
0,논현로6길,37.473561,127.050895
1,논현로18길,37.477885,127.046338
2,개포로20길,37.477040,127.048356
3,선릉로12길,37.483899,127.060991
4,논현로8길,37.474467,127.051458


In [10]:
# 주소 검색 함수 정의
def category_count(code, x, y):
    url = "https://dapi.kakao.com/v2/local/search/category.json?category_group_code=" + code + "&x=" + x + "&y=" + y + "&radius=300"
    result = requests.get(urlparse(url).geturl(),
                          headers={"Authorization":"KakaoAK *****"})
    json_obj = result.json()
    val = []
    for document in json_obj['documents']:
        # 값이 NoneType 일 경우의 오류 방지
        try:
            val.append([x, y, document['place_name'], document['road_address_name'], document['x'], document['y']])
        except:
            val.append([0, 0, 0, 0, 0, 0])
    return val

In [15]:
sample = category_count('MT1', '127.060991', '37.483899')   # 경도, 위도

In [16]:
sample2 = pd.DataFrame(sample, columns=['경도','위도','장소명','도로명','편의시설_경도','편의시설_위도'])
sample2.head()

,경도,위도,장소명,도로명,편의시설_경도,편의시설_위도
0,127.060991,37.483899,GS수퍼마켓 개포블레스티지점,서울 강남구 선릉로 8,127.06221728439263,37.4821587758915


In [17]:
# 주소 검색 함수 반복 적용
list1 = []
for i in tqdm(range(len(data))):
    time.sleep(0.1)
    list1.append(category_count('MT1', str(data['경도'][i]), str(data['위도'][i])))

100%|████████████████████████████████████████████████████████████████████████████| 12342/12342 [46:01<00:00,  4.47it/s]


In [18]:
list1

[[],
 [],
 [],
 [['127.060991290466',
   '37.4838993527963',
   'GS수퍼마켓 개포블레스티지점',
   '서울 강남구 선릉로 8',
   '127.06221728439263',
   '37.4821587758915']],
 [],
 [['127.060693844398',
   '37.4840747521106',
   'GS수퍼마켓 개포블레스티지점',
   '서울 강남구 선릉로 8',
   '127.06221728439263',
   '37.4821587758915']],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [['127.06111847254499',
   '37.4836015938737',
   'GS수퍼마켓 개포블레스티지점',
   '서울 강남구 선릉로 8',
   '127.06221728439263',
   '37.4821587758915']],
 [],
 [],
 [],
 [],
 [],
 [['127.06198259187599',
   '37.4832847996201',
   'GS수퍼마켓 개포블레스티지점',
   '서울 강남구 선릉로 8',
   '127.06221728439263',
   '37.4821587758915']],
 [],
 [],
 [['127.06007757611201',
   '37.4839988417672',
   'GS수퍼마켓 개포블레스티지점',
   '서울 강남구 선릉로 8',
   '127.06221728439263',
   '37.4821587758915']],
 [['127.06133252779499',
   '37.4844601439865',
   'GS수퍼마켓 개포블레스티지점',
   '서울 강남구 선릉로 8',
   '127.06221728439263',
   '37.4821587758915']],
 [],
 [],
 [['127.062105811819',
   '37.48298

In [19]:
list2 = [i for j in list1 for i in j]
list2

[['127.060991290466',
  '37.4838993527963',
  'GS수퍼마켓 개포블레스티지점',
  '서울 강남구 선릉로 8',
  '127.06221728439263',
  '37.4821587758915'],
 ['127.060693844398',
  '37.4840747521106',
  'GS수퍼마켓 개포블레스티지점',
  '서울 강남구 선릉로 8',
  '127.06221728439263',
  '37.4821587758915'],
 ['127.06111847254499',
  '37.4836015938737',
  'GS수퍼마켓 개포블레스티지점',
  '서울 강남구 선릉로 8',
  '127.06221728439263',
  '37.4821587758915'],
 ['127.06198259187599',
  '37.4832847996201',
  'GS수퍼마켓 개포블레스티지점',
  '서울 강남구 선릉로 8',
  '127.06221728439263',
  '37.4821587758915'],
 ['127.06007757611201',
  '37.4839988417672',
  'GS수퍼마켓 개포블레스티지점',
  '서울 강남구 선릉로 8',
  '127.06221728439263',
  '37.4821587758915'],
 ['127.06133252779499',
  '37.4844601439865',
  'GS수퍼마켓 개포블레스티지점',
  '서울 강남구 선릉로 8',
  '127.06221728439263',
  '37.4821587758915'],
 ['127.062105811819',
  '37.482987131784895',
  'GS수퍼마켓 개포블레스티지점',
  '서울 강남구 선릉로 8',
  '127.06221728439263',
  '37.4821587758915'],
 ['127.03278402308001',
  '37.5124619067575',
  '홈플러스익스프레스 학동역점',
  '서울 강남구 학동로 

In [20]:
data2 = pd.DataFrame(list2, columns=['경도','위도','장소명','도로명','편의시설_경도','편의시설_위도'])
print(data2.shape)
data2.head()

(3663, 6)


,경도,위도,장소명,도로명,편의시설_경도,편의시설_위도
0,127.060991290466,37.4838993527963,GS수퍼마켓 개포블레스티지점,서울 강남구 선릉로 8,127.06221728439263,37.4821587758915
1,127.060693844398,37.4840747521106,GS수퍼마켓 개포블레스티지점,서울 강남구 선릉로 8,127.06221728439263,37.4821587758915
2,127.06111847254499,37.4836015938737,GS수퍼마켓 개포블레스티지점,서울 강남구 선릉로 8,127.06221728439263,37.4821587758915
3,127.06198259187599,37.4832847996201,GS수퍼마켓 개포블레스티지점,서울 강남구 선릉로 8,127.06221728439263,37.4821587758915
4,127.06007757611201,37.4839988417672,GS수퍼마켓 개포블레스티지점,서울 강남구 선릉로 8,127.06221728439263,37.4821587758915


In [21]:
data2['주택형태'] = '단독다가구'

In [22]:
data2.head()

,경도,위도,장소명,도로명,편의시설_경도,편의시설_위도,주택형태
0,127.060991290466,37.4838993527963,GS수퍼마켓 개포블레스티지점,서울 강남구 선릉로 8,127.06221728439263,37.4821587758915,단독다가구
1,127.060693844398,37.4840747521106,GS수퍼마켓 개포블레스티지점,서울 강남구 선릉로 8,127.06221728439263,37.4821587758915,단독다가구
2,127.06111847254499,37.4836015938737,GS수퍼마켓 개포블레스티지점,서울 강남구 선릉로 8,127.06221728439263,37.4821587758915,단독다가구
3,127.06198259187599,37.4832847996201,GS수퍼마켓 개포블레스티지점,서울 강남구 선릉로 8,127.06221728439263,37.4821587758915,단독다가구
4,127.06007757611201,37.4839988417672,GS수퍼마켓 개포블레스티지점,서울 강남구 선릉로 8,127.06221728439263,37.4821587758915,단독다가구


In [29]:
data2.tail()

,경도,위도,장소명,도로명,편의시설_경도,편의시설_위도,주택형태
3658,127.013117953109,37.4858295504798,홈플러스익스프레스 서초2점,서울 서초구 서초중앙로 56,127.015449862122,37.4868756093909,단독다가구
3659,126.905380298881,37.516757160043895,이마트 영등포점,서울 영등포구 영중로 15,126.903331121684,37.5174496335566,단독다가구
3660,127.00254257684999,37.5328356664747,롯데슈퍼 한남가맹점,서울 용산구 대사관로34길 26,127.005645582603,37.5320051061339,단독다가구
3661,126.970974154065,37.572784541420496,서서울농협하나로마트 사직점,,126.96999114989003,37.57470692847596,단독다가구
3662,127.01047746532,37.5707241856781,노브랜드 동대문두타몰점,서울 중구 장충단로 275,127.00868533599005,37.568750340187734,단독다가구


In [23]:
data2.to_csv('편의시설/단독다가구_대형마트추가.csv', index=False)

## 1.2 연립

In [14]:
data3 = pd.read_csv('data/연립_위도경도.csv')
print(data3.shape)
data3.head()

(82435, 3)


,도로명,위도,경도
0,논현로 10,37.472388,127.050511
1,논현로2길 66,37.474544,127.055084
2,논현로8길 62-1,37.474544,127.055084
3,논현로8길 63,37.475313,127.053546
4,개포로26길 16,37.478034,127.049109


In [25]:
# 주소 검색 함수 반복 적용
list3 = []
for i in tqdm(range(len(data3))):
    time.sleep(0.1)
    list3.append(category_count('MT1', str(data3['경도'][i]), str(data3['위도'][i])))

 44%|███████████████████████████████▌                                        | 36103/82435 [3:06:44<3:59:38,  3.22it/s]


KeyError: 'documents'

In [26]:
# API 쿼터 제한으로 오류 중단 -> 일단 데이터프레임으로 저장
list3

[[],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [['127.061758559555',
   '37.482947218924',
   'GS수퍼마켓 개포블레스티지점',
   '서울 강남구 선릉로 8',
   '127.06221728439263',
   '37.4821587758915']],
 [['127.062784134247',
   '37.483589457891995',
   'GS수퍼마켓 개포블레스티지점',
   '서울 강남구 선릉로 8',
   '127.06221728439263',
   '37.4821587758915']],
 [['127.060774393715',
   '37.4834343639804',
   'GS수퍼마켓 개포블레스티지점',
   '서울 강남구 선릉로 8',
   '127.06221728439263',
   '37.4821587758915']],
 [['127.060373056666',
   '37.48362756576179',
   'GS수퍼마켓 개포블레스티지점',
   '서울 강남구 선릉로 8',
   '127.06221728439263',
   '37.4821587758915']],
 [['127.060974874536',
   '37.484012077425795',
   'GS수퍼마켓 개포블레스티지점',
   '서울 강남구 선릉로 8',
   '127.06221728439263',
   '37.4821587758915']],
 [['127.060694540667',
   '37.483957080051105',
   'GS수퍼마켓 개포블레스티지점',
   '서울 강남구 선릉로 8',
   '127.06221728439263',
   '37.4821587758915']],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [['127.060486695556',
   '37.4841992876044',
   'GS수퍼마켓 개포블레스티지점',
   '서울 강남구 선릉로

In [27]:
list4 = [i for j in list3 for i in j]
list4

[['127.061758559555',
  '37.482947218924',
  'GS수퍼마켓 개포블레스티지점',
  '서울 강남구 선릉로 8',
  '127.06221728439263',
  '37.4821587758915'],
 ['127.062784134247',
  '37.483589457891995',
  'GS수퍼마켓 개포블레스티지점',
  '서울 강남구 선릉로 8',
  '127.06221728439263',
  '37.4821587758915'],
 ['127.060774393715',
  '37.4834343639804',
  'GS수퍼마켓 개포블레스티지점',
  '서울 강남구 선릉로 8',
  '127.06221728439263',
  '37.4821587758915'],
 ['127.060373056666',
  '37.48362756576179',
  'GS수퍼마켓 개포블레스티지점',
  '서울 강남구 선릉로 8',
  '127.06221728439263',
  '37.4821587758915'],
 ['127.060974874536',
  '37.484012077425795',
  'GS수퍼마켓 개포블레스티지점',
  '서울 강남구 선릉로 8',
  '127.06221728439263',
  '37.4821587758915'],
 ['127.060694540667',
  '37.483957080051105',
  'GS수퍼마켓 개포블레스티지점',
  '서울 강남구 선릉로 8',
  '127.06221728439263',
  '37.4821587758915'],
 ['127.060486695556',
  '37.4841992876044',
  'GS수퍼마켓 개포블레스티지점',
  '서울 강남구 선릉로 8',
  '127.06221728439263',
  '37.4821587758915'],
 ['127.06131781719',
  '37.4836153661169',
  'GS수퍼마켓 개포블레스티지점',
  '서울 강남구 선릉로 8',
  

In [28]:
data4 = pd.DataFrame(list4, columns=['경도','위도','장소명','도로명','편의시설_경도','편의시설_위도'])
print(data4.shape)
data4.head()

(10123, 6)


,경도,위도,장소명,도로명,편의시설_경도,편의시설_위도
0,127.061758559555,37.482947218924,GS수퍼마켓 개포블레스티지점,서울 강남구 선릉로 8,127.06221728439263,37.4821587758915
1,127.062784134247,37.483589457891995,GS수퍼마켓 개포블레스티지점,서울 강남구 선릉로 8,127.06221728439263,37.4821587758915
2,127.060774393715,37.4834343639804,GS수퍼마켓 개포블레스티지점,서울 강남구 선릉로 8,127.06221728439263,37.4821587758915
3,127.060373056666,37.48362756576179,GS수퍼마켓 개포블레스티지점,서울 강남구 선릉로 8,127.06221728439263,37.4821587758915
4,127.060974874536,37.484012077425795,GS수퍼마켓 개포블레스티지점,서울 강남구 선릉로 8,127.06221728439263,37.4821587758915


In [30]:
data4['주택형태'] = '연립'

In [31]:
data4.head()

,경도,위도,장소명,도로명,편의시설_경도,편의시설_위도,주택형태
0,127.061758559555,37.482947218924,GS수퍼마켓 개포블레스티지점,서울 강남구 선릉로 8,127.06221728439263,37.4821587758915,연립
1,127.062784134247,37.483589457891995,GS수퍼마켓 개포블레스티지점,서울 강남구 선릉로 8,127.06221728439263,37.4821587758915,연립
2,127.060774393715,37.4834343639804,GS수퍼마켓 개포블레스티지점,서울 강남구 선릉로 8,127.06221728439263,37.4821587758915,연립
3,127.060373056666,37.48362756576179,GS수퍼마켓 개포블레스티지점,서울 강남구 선릉로 8,127.06221728439263,37.4821587758915,연립
4,127.060974874536,37.484012077425795,GS수퍼마켓 개포블레스티지점,서울 강남구 선릉로 8,127.06221728439263,37.4821587758915,연립


In [32]:
data4.tail()

,경도,위도,장소명,도로명,편의시설_경도,편의시설_위도,주택형태
10118,126.89414193246901,37.5389796534264,노브랜드 영등포양평점,서울 영등포구 양평로 135,126.89268548684787,37.53818960363232,연립
10119,126.89126280072901,37.537313881767,노브랜드 영등포양평점,서울 영등포구 양평로 135,126.89268548684787,37.53818960363232,연립
10120,126.89172742011499,37.536815064545294,노브랜드 영등포양평점,서울 영등포구 양평로 135,126.89268548684787,37.53818960363232,연립
10121,126.893458687323,37.5390183225669,노브랜드 영등포양평점,서울 영등포구 양평로 135,126.89268548684787,37.53818960363232,연립
10122,126.89436002729799,37.5390188620186,노브랜드 영등포양평점,서울 영등포구 양평로 135,126.89268548684787,37.53818960363232,연립


In [33]:
data4.to_csv('편의시설/연립_대형마트추가(1).csv', index=False)

In [13]:
# 이어서 주소 검색 함수 적용
data5 = data3[36102:].reset_index(drop=True)
print(data5.shape)
data5.head()

(46333, 3)


,도로명,위도,경도
0,버드나루로3길 1,37.520971,126.911969
1,국회대로56길 35,37.522287,126.910874
2,국회대로54길 41-14,37.522021,126.910331
3,국회대로56길 58,37.520323,126.911800
4,국회대로56길 37,37.522150,126.910935


In [14]:
list5 = []
for i in tqdm(range(len(data5))):
    time.sleep(0.1)
    list5.append(category_count('MT1', str(data5['경도'][i]), str(data5['위도'][i])))

100%|██████████████████████████████████████████████████████████████████████████| 46333/46333 [2:40:08<00:00,  4.82it/s]


In [15]:
list5

[[],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [['126.902629968382',
   '37.5188894010257',
   '이마트 영등포점',
   '서울 영등포구 영중로 15',
   '126.903331121684',
   '37.5174496335566']],
 [['126.902499110716',
   '37.518874066127',
   '이마트 영등포점',
   '서울 영등포구 영중로 15',
   '126.903331121684',
   '37.5174496335566']],
 [['126.902598462901',
   '37.5190215521725',
   '이마트 영등포점',
   '서울 영등포구 영중로 15',
   '126.903331121684',
   '37.5174496335566']],
 [['126.90146803173',
   '37.518966103874',
   '이마트 영등포점',
   '서울 영등포구 영중로 15',
   '126.903331121684',
   '37.5174496335566']],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [['126.906273087949',
   '37.5270875947617',
   '롯데빅마켓 영등포점',
   '서울 영등포구 영중로 125',
   '126.904611994126',
   '37.5277977858063']],
 [['126.903752693076',
   '37.5262650254161',
   '롯데빅마켓 영등포점',
   '서울 영등포구 영중로 125',
   '126.9046119

In [16]:
list6 = [i for j in list5 for i in j]
list6

[['126.902629968382',
  '37.5188894010257',
  '이마트 영등포점',
  '서울 영등포구 영중로 15',
  '126.903331121684',
  '37.5174496335566'],
 ['126.902499110716',
  '37.518874066127',
  '이마트 영등포점',
  '서울 영등포구 영중로 15',
  '126.903331121684',
  '37.5174496335566'],
 ['126.902598462901',
  '37.5190215521725',
  '이마트 영등포점',
  '서울 영등포구 영중로 15',
  '126.903331121684',
  '37.5174496335566'],
 ['126.90146803173',
  '37.518966103874',
  '이마트 영등포점',
  '서울 영등포구 영중로 15',
  '126.903331121684',
  '37.5174496335566'],
 ['126.906273087949',
  '37.5270875947617',
  '롯데빅마켓 영등포점',
  '서울 영등포구 영중로 125',
  '126.904611994126',
  '37.5277977858063'],
 ['126.903752693076',
  '37.5262650254161',
  '롯데빅마켓 영등포점',
  '서울 영등포구 영중로 125',
  '126.904611994126',
  '37.5277977858063'],
 ['126.90605743564899',
  '37.5264765442602',
  '롯데빅마켓 영등포점',
  '서울 영등포구 영중로 125',
  '126.904611994126',
  '37.5277977858063'],
 ['126.90658704528799',
  '37.526254777297204',
  '롯데빅마켓 영등포점',
  '서울 영등포구 영중로 125',
  '126.904611994126',
  '37.5277977858063'],
 

In [17]:
data6 = pd.DataFrame(list6, columns=['경도','위도','장소명','도로명','편의시설_경도','편의시설_위도'])
print(data6.shape)
data6.head()

(12901, 6)


,경도,위도,장소명,도로명,편의시설_경도,편의시설_위도
0,126.902629968382,37.5188894010257,이마트 영등포점,서울 영등포구 영중로 15,126.903331121684,37.5174496335566
1,126.902499110716,37.518874066127,이마트 영등포점,서울 영등포구 영중로 15,126.903331121684,37.5174496335566
2,126.902598462901,37.5190215521725,이마트 영등포점,서울 영등포구 영중로 15,126.903331121684,37.5174496335566
3,126.90146803173,37.518966103874,이마트 영등포점,서울 영등포구 영중로 15,126.903331121684,37.5174496335566
4,126.906273087949,37.5270875947617,롯데빅마켓 영등포점,서울 영등포구 영중로 125,126.904611994126,37.5277977858063


In [18]:
data6['주택형태'] = '연립'

In [19]:
data6.head()

,경도,위도,장소명,도로명,편의시설_경도,편의시설_위도,주택형태
0,126.902629968382,37.5188894010257,이마트 영등포점,서울 영등포구 영중로 15,126.903331121684,37.5174496335566,연립
1,126.902499110716,37.518874066127,이마트 영등포점,서울 영등포구 영중로 15,126.903331121684,37.5174496335566,연립
2,126.902598462901,37.5190215521725,이마트 영등포점,서울 영등포구 영중로 15,126.903331121684,37.5174496335566,연립
3,126.90146803173,37.518966103874,이마트 영등포점,서울 영등포구 영중로 15,126.903331121684,37.5174496335566,연립
4,126.906273087949,37.5270875947617,롯데빅마켓 영등포점,서울 영등포구 영중로 125,126.904611994126,37.5277977858063,연립


In [20]:
data6.tail()

,경도,위도,장소명,도로명,편의시설_경도,편의시설_위도,주택형태
12896,127.096410578097,37.6034858246385,동서울농협하나로마트 신내점,서울 중랑구 신내로 74,127.095659826787,37.6059282980536,연립
12897,127.077287608867,37.59690500617221,홈플러스익스프레스 태릉점,서울 중랑구 동일로129길 35,127.077505728865,37.5992238335536,연립
12898,127.080171771637,37.603906932258795,서울원예농협하나로마트,서울 중랑구 동일로 842,127.078966481942,37.6056162919763,연립
12899,127.08019851188801,37.603498044813705,서울원예농협하나로마트,서울 중랑구 동일로 842,127.078966481942,37.6056162919763,연립
12900,127.074939090163,37.598596509446004,홈플러스익스프레스 태릉점,서울 중랑구 동일로129길 35,127.077505728865,37.5992238335536,연립


In [21]:
data6.to_csv('편의시설/연립_대형마트추가(2).csv', index=False)

## 1.3 오피스텔

In [22]:
data7 = pd.read_csv('data/오피스텔_위도경도.csv')
print(data7.shape)
data7.head()

(3462, 3)


,도로명,위도,경도
0,개포로 213,37.478745,127.046113
1,개포로 623,37.494341,127.079648
2,논현로18길 10-10,37.477252,127.046084
3,선릉로129길 3-4,37.516104,127.041071
4,도산대로46길 11,37.521292,127.036652


In [23]:
list7 = []
for i in tqdm(range(len(data7))):
    time.sleep(0.1)
    list7.append(category_count('MT1', str(data7['경도'][i]), str(data7['위도'][i])))

100%|██████████████████████████████████████████████████████████████████████████████| 3462/3462 [11:55<00:00,  4.84it/s]


In [24]:
list7

[[],
 [['127.07964775319401',
   '37.494341198165',
   '이마트에브리데이 개포동점',
   '서울 강남구 개포로 623',
   '127.0798902907556',
   '37.49400270635865']],
 [],
 [],
 [],
 [],
 [],
 [],
 [['127.03607608067',
   '37.5125390642313',
   '롯데슈퍼 논현점',
   '서울 강남구 언주로122길 25',
   '127.03866975955695',
   '37.512952615393765']],
 [],
 [],
 [],
 [],
 [],
 [],
 [['127.060358414855',
   '37.4936757916501',
   '노브랜드 강남대치역점',
   '서울 강남구 남부순환로 2947',
   '127.062583748312',
   '37.4945258149659'],
  ['127.060358414855',
   '37.4936757916501',
   'GS수퍼마켓 강남대치점',
   '서울 강남구 남부순환로 2927',
   '127.06033483167253',
   '37.49345748971275'],
  ['127.060358414855',
   '37.4936757916501',
   '홈플러스익스프레스 대치역점',
   '서울 강남구 삼성로 155',
   '127.062940561983',
   '37.4939021286529']],
 [['127.053831329394',
   '37.50503117269321',
   '롯데마이슈퍼 대치2점',
   '서울 강남구 테헤란로78길 16',
   '127.05551375517567',
   '37.50502021266819']],
 [['127.053801125765',
   '37.5045490585229',
   '롯데마이슈퍼 대치2점',
   '서울 강남구 테헤란로78길 16',
   '127.05551375517567'

In [25]:
list8 = [i for j in list7 for i in j]
list8

[['127.07964775319401',
  '37.494341198165',
  '이마트에브리데이 개포동점',
  '서울 강남구 개포로 623',
  '127.0798902907556',
  '37.49400270635865'],
 ['127.03607608067',
  '37.5125390642313',
  '롯데슈퍼 논현점',
  '서울 강남구 언주로122길 25',
  '127.03866975955695',
  '37.512952615393765'],
 ['127.060358414855',
  '37.4936757916501',
  '노브랜드 강남대치역점',
  '서울 강남구 남부순환로 2947',
  '127.062583748312',
  '37.4945258149659'],
 ['127.060358414855',
  '37.4936757916501',
  'GS수퍼마켓 강남대치점',
  '서울 강남구 남부순환로 2927',
  '127.06033483167253',
  '37.49345748971275'],
 ['127.060358414855',
  '37.4936757916501',
  '홈플러스익스프레스 대치역점',
  '서울 강남구 삼성로 155',
  '127.062940561983',
  '37.4939021286529'],
 ['127.053831329394',
  '37.50503117269321',
  '롯데마이슈퍼 대치2점',
  '서울 강남구 테헤란로78길 16',
  '127.05551375517567',
  '37.50502021266819'],
 ['127.053801125765',
  '37.5045490585229',
  '롯데마이슈퍼 대치2점',
  '서울 강남구 테헤란로78길 16',
  '127.05551375517567',
  '37.50502021266819'],
 ['127.055542809861',
  '37.497829361835606',
  '노브랜드 강남한티역점',
  '서울 강남구 선릉로 318',
 

In [26]:
data8 = pd.DataFrame(list8, columns=['경도','위도','장소명','도로명','편의시설_경도','편의시설_위도'])
print(data8.shape)
data8.head()

(1661, 6)


,경도,위도,장소명,도로명,편의시설_경도,편의시설_위도
0,127.07964775319401,37.494341198165,이마트에브리데이 개포동점,서울 강남구 개포로 623,127.0798902907556,37.49400270635865
1,127.03607608067,37.5125390642313,롯데슈퍼 논현점,서울 강남구 언주로122길 25,127.03866975955695,37.512952615393765
2,127.060358414855,37.4936757916501,노브랜드 강남대치역점,서울 강남구 남부순환로 2947,127.062583748312,37.4945258149659
3,127.060358414855,37.4936757916501,GS수퍼마켓 강남대치점,서울 강남구 남부순환로 2927,127.06033483167253,37.49345748971275
4,127.060358414855,37.4936757916501,홈플러스익스프레스 대치역점,서울 강남구 삼성로 155,127.062940561983,37.4939021286529


In [27]:
data8['주택형태'] = '오피스텔'

In [28]:
data8.head()

,경도,위도,장소명,도로명,편의시설_경도,편의시설_위도,주택형태
0,127.07964775319401,37.494341198165,이마트에브리데이 개포동점,서울 강남구 개포로 623,127.0798902907556,37.49400270635865,오피스텔
1,127.03607608067,37.5125390642313,롯데슈퍼 논현점,서울 강남구 언주로122길 25,127.03866975955695,37.512952615393765,오피스텔
2,127.060358414855,37.4936757916501,노브랜드 강남대치역점,서울 강남구 남부순환로 2947,127.062583748312,37.4945258149659,오피스텔
3,127.060358414855,37.4936757916501,GS수퍼마켓 강남대치점,서울 강남구 남부순환로 2927,127.06033483167253,37.49345748971275,오피스텔
4,127.060358414855,37.4936757916501,홈플러스익스프레스 대치역점,서울 강남구 삼성로 155,127.062940561983,37.4939021286529,오피스텔


In [29]:
data8.tail()

,경도,위도,장소명,도로명,편의시설_경도,편의시설_위도,주택형태
1656,127.077632111372,37.5927999533509,롯데슈퍼 상봉점,서울 중랑구 망우로32길 23,127.07689165330059,37.592863417369124,오피스텔
1657,127.08741628398701,37.59520254398279,동서울농협,서울 중랑구 망우로 288-1,127.084498976668,37.5958394169252,오피스텔
1658,127.086397221943,37.595432596612795,동서울농협,서울 중랑구 망우로 288-1,127.084498976668,37.5958394169252,오피스텔
1659,127.091116225847,37.5944900348356,코스트코 상봉점,서울 중랑구 망우로 336,127.090212893898,37.5968650207457,오피스텔
1660,127.07533835216499,37.594423584999205,롯데슈퍼 상봉점,서울 중랑구 망우로32길 23,127.07689165330059,37.592863417369124,오피스텔


In [30]:
data8.to_csv('편의시설/오피스텔_대형마트추가.csv', index=False)

# 2. 데이터 통합

In [5]:
단독다가구 = pd.read_csv('편의시설/단독다가구_대형마트추가.csv')
print(단독다가구.shape)
단독다가구.head()

(3663, 7)


,경도,위도,장소명,도로명,편의시설_경도,편의시설_위도,주택형태
0,127.060991,37.483899,GS수퍼마켓 개포블레스티지점,서울 강남구 선릉로 8,127.062217,37.482159,단독다가구
1,127.060694,37.484075,GS수퍼마켓 개포블레스티지점,서울 강남구 선릉로 8,127.062217,37.482159,단독다가구
2,127.061118,37.483602,GS수퍼마켓 개포블레스티지점,서울 강남구 선릉로 8,127.062217,37.482159,단독다가구
3,127.061983,37.483285,GS수퍼마켓 개포블레스티지점,서울 강남구 선릉로 8,127.062217,37.482159,단독다가구
4,127.060078,37.483999,GS수퍼마켓 개포블레스티지점,서울 강남구 선릉로 8,127.062217,37.482159,단독다가구


In [6]:
연립1 = pd.read_csv('편의시설/연립_대형마트추가(1).csv')
print(연립1.shape)
연립1.head()

(10123, 7)


,경도,위도,장소명,도로명,편의시설_경도,편의시설_위도,주택형태
0,127.061759,37.482947,GS수퍼마켓 개포블레스티지점,서울 강남구 선릉로 8,127.062217,37.482159,연립
1,127.062784,37.483589,GS수퍼마켓 개포블레스티지점,서울 강남구 선릉로 8,127.062217,37.482159,연립
2,127.060774,37.483434,GS수퍼마켓 개포블레스티지점,서울 강남구 선릉로 8,127.062217,37.482159,연립
3,127.060373,37.483628,GS수퍼마켓 개포블레스티지점,서울 강남구 선릉로 8,127.062217,37.482159,연립
4,127.060975,37.484012,GS수퍼마켓 개포블레스티지점,서울 강남구 선릉로 8,127.062217,37.482159,연립


In [7]:
연립2 = pd.read_csv('편의시설/연립_대형마트추가(2).csv')
print(연립2.shape)
연립2.head()

(12901, 7)


,경도,위도,장소명,도로명,편의시설_경도,편의시설_위도,주택형태
0,126.902630,37.518889,이마트 영등포점,서울 영등포구 영중로 15,126.903331,37.517450,연립
1,126.902499,37.518874,이마트 영등포점,서울 영등포구 영중로 15,126.903331,37.517450,연립
2,126.902598,37.519022,이마트 영등포점,서울 영등포구 영중로 15,126.903331,37.517450,연립
3,126.901468,37.518966,이마트 영등포점,서울 영등포구 영중로 15,126.903331,37.517450,연립
4,126.906273,37.527088,롯데빅마켓 영등포점,서울 영등포구 영중로 125,126.904612,37.527798,연립


In [8]:
오피스텔 = pd.read_csv('편의시설/오피스텔_대형마트추가.csv')
print(오피스텔.shape)
오피스텔.head()

(1661, 7)


,경도,위도,장소명,도로명,편의시설_경도,편의시설_위도,주택형태
0,127.079648,37.494341,이마트에브리데이 개포동점,서울 강남구 개포로 623,127.079890,37.494003,오피스텔
1,127.036076,37.512539,롯데슈퍼 논현점,서울 강남구 언주로122길 25,127.038670,37.512953,오피스텔
2,127.060358,37.493676,노브랜드 강남대치역점,서울 강남구 남부순환로 2947,127.062584,37.494526,오피스텔
3,127.060358,37.493676,GS수퍼마켓 강남대치점,서울 강남구 남부순환로 2927,127.060335,37.493457,오피스텔
4,127.060358,37.493676,홈플러스익스프레스 대치역점,서울 강남구 삼성로 155,127.062941,37.493902,오피스텔


In [23]:
total_data = pd.concat([단독다가구, 연립1, 연립2, 오피스텔], axis=0).reset_index(drop=True)
print(total_data.shape)
total_data.head()

(28348, 7)


,경도,위도,장소명,도로명,편의시설_경도,편의시설_위도,주택형태
0,127.060991,37.483899,GS수퍼마켓 개포블레스티지점,서울 강남구 선릉로 8,127.062217,37.482159,단독다가구
1,127.060694,37.484075,GS수퍼마켓 개포블레스티지점,서울 강남구 선릉로 8,127.062217,37.482159,단독다가구
2,127.061118,37.483602,GS수퍼마켓 개포블레스티지점,서울 강남구 선릉로 8,127.062217,37.482159,단독다가구
3,127.061983,37.483285,GS수퍼마켓 개포블레스티지점,서울 강남구 선릉로 8,127.062217,37.482159,단독다가구
4,127.060078,37.483999,GS수퍼마켓 개포블레스티지점,서울 강남구 선릉로 8,127.062217,37.482159,단독다가구


In [24]:
total_data['주택형태'].unique()

array(['단독다가구', '연립', '오피스텔'], dtype=object)

In [25]:
len(total_data['장소명'].unique())

485

In [26]:
total_data['도로명'].value_counts()

서울 송파구 백제고분로 207         421
서울 강서구 까치산로 106          343
서울 은평구 갈현로 34            317
서울 구로구 개봉로17길 34         311
서울 은평구 갈현로 10            299
                        ... 
경기 의정부시 오목로205번길 50        1
서울 양천구 목동로 212             1
전북 정읍시 벚꽃로 85              1
경북 경산시 용성면 원효로 1418-1      1
경남 김해시 가락로 83              1
Name: 도로명, Length: 479, dtype: int64

In [27]:
# 추후 삭제 예정
not_seoul = total_data[total_data['도로명'].str.contains('서울') == False]
print(not_seoul.shape)
not_seoul

(252, 7)


,경도,위도,장소명,도로명,편의시설_경도,편의시설_위도,주택형태
259,127.171596,37.547143,하나로마트 서부농협초이지점,경기 하남시 초이로 160-1,127.172045,37.544974,단독다가구
260,127.171420,37.547618,하나로마트 서부농협초이지점,경기 하남시 초이로 160-1,127.172045,37.544974,단독다가구
762,129.319437,35.555231,노브랜드 울산성남점,울산 중구 젊음의2거리 33,129.320489,35.553074,단독다가구
804,127.224858,37.002323,대덕농협하나로마트 내리점,경기 안성시 대덕면 중앙대학로 126,127.224664,37.000579,단독다가구
805,127.227485,36.999701,대덕농협하나로마트 내리점,경기 안성시 대덕면 중앙대학로 126,127.224664,37.000579,단독다가구
...,...,...,...,...,...,...,...
28096,126.780206,37.483030,부천시흥원예농협 하나로마트 로컬푸드점,경기 부천시 심곡로15번길 32,126.778361,37.482030,오피스텔
28106,126.837711,35.578351,롯데마트 정읍점,전북 정읍시 벚꽃로 85,126.839784,35.579343,오피스텔
28262,128.263654,34.637236,통영농협하나로마트 욕지점,경남 통영시 욕지면 욕지일주로 95-76,128.264490,34.635678,오피스텔
28277,126.755529,37.710345,GS수퍼마켓 파주파크타운점,경기 파주시 한빛로 20,126.755528,37.710346,오피스텔


In [28]:
total_data.to_csv('편의시설/300m내_대형마트.csv', index=False)